# 06 - Model Comparison (Full 2025 Test)

**Purpose:** Comprehensive evaluation of all trained models on the complete 2025 season (unseen data).

**MLOps Best Practices:**
- Consistent model colors and styling across all plots
- All metrics saved to CSV for reproducibility
- Bootstrap confidence intervals for statistical rigor
- Publication-ready plots (HTML + PNG)

**Sections:**
1. Configuration & Setup
2. Data Loading (2025 Test Set)
3. Model Loading & Inference
4. **Comprehensive Numerical Metrics** (all metrics, no plots)
5. Model Comparison Plots
6. Per-Model Diagnostic Plots
7. Error Analysis by Category
8. Temporal Analysis (by Round)

**Output:** `reports/notebooks/06_model_comparison_full/`

---
## 1. Configuration & Setup

In [ ]:
import sys
from pathlib import Path

ROOT = Path.cwd().resolve().parent if Path.cwd().name == "notebooks" else Path.cwd().resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from _common import load_dataset, prepare_features
from src.split import SplitConfig
from src.eval import compute_full_metrics
from src.plots import save_plot

# Reproducibility
SEED = 42
np.random.seed(SEED)

print(f"Project root: {ROOT}")

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Output directory
OUTPUT_DIR = ROOT / "reports" / "notebooks" / "06_model_comparison_full"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Model paths
MODELS_DIR = ROOT / "reports" / "models"
MODEL_NAMES = ["Linear", "XGBoost", "Deep MLP"]

# Consistent color scheme for all plots
MODEL_COLORS = {
    "Linear": "#636EFA",      # Blue
    "XGBoost": "#EF553B",     # Red
    "Deep MLP": "#00CC96",    # Green
}

# Plot styling
PLOT_TEMPLATE = "plotly_white"
PLOT_FONT_SIZE = 13
PLOT_HEIGHT = 480
PLOT_WIDTH = 800

# Bootstrap settings
BOOTSTRAP_N = 500
BOOTSTRAP_CI = 95  # percent

print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
# =============================================================================
# PLOTTING UTILITIES
# =============================================================================

def style_figure(fig, title=None, height=PLOT_HEIGHT, width=None):
    """Apply consistent styling to all figures."""
    fig.update_layout(
        template=PLOT_TEMPLATE,
        height=height,
        width=width,
        title=dict(text=title or fig.layout.title.text, font=dict(size=16)),
        font=dict(size=PLOT_FONT_SIZE),
        legend=dict(font=dict(size=12)),
    )
    fig.update_xaxes(
        showline=True, linewidth=1, linecolor="black", mirror=True,
        ticks="outside", showgrid=True, gridcolor="rgba(0,0,0,0.08)",
    )
    fig.update_yaxes(
        showline=True, linewidth=1, linecolor="black", mirror=True,
        ticks="outside", showgrid=True, gridcolor="rgba(0,0,0,0.08)",
    )
    return fig


def save_and_show(fig, name):
    """Save figure to HTML and PNG, then display."""
    save_plot(fig, OUTPUT_DIR / name)
    fig.show()

---
## 2. Data Loading (2025 Test Set)

In [ ]:
# Load dataset with full 2025 as test (test_rounds=None means all available)
split_config = SplitConfig(test_rounds=None)
df, metadata = load_dataset()
train_df, val_df, trainval_df, test_df, features = prepare_features(
    df, metadata, split_config=split_config
)

# Extract test arrays
X_test = test_df[features]
y_test = test_df["LapTimeSeconds"].to_numpy()

print("=" * 60)
print("TEST SET SUMMARY (2025 Season - Unseen Data)")
print("=" * 60)
print(f"Total laps:     {len(test_df):,}")
print(f"Features:       {len(features)}")
print(f"Season(s):      {sorted(test_df['Season'].unique())}")

In [ ]:
# Show test set composition
test_summary = (
    test_df.groupby(["Season", "RoundNumber", "EventName"])
    .size()
    .reset_index(name="Laps")
    .sort_values(["Season", "RoundNumber"])
)
print(f"\nRaces in test set: {len(test_summary)}")
print(f"Rounds: {test_summary['RoundNumber'].tolist()}")
test_summary

---
## 3. Model Loading & Inference

In [ ]:
# Load all saved models
models = {}
for name in MODEL_NAMES:
    path = MODELS_DIR / f"{name.lower().replace(' ', '_')}.joblib"
    if path.exists():
        models[name] = joblib.load(path)
        print(f"Loaded: {name} from {path.name}")
    else:
        print(f"WARNING: {name} not found at {path}")

if not models:
    raise FileNotFoundError(f"No models found in {MODELS_DIR}. Run training notebooks first.")

print(f"\nModels loaded: {list(models.keys())}")

In [ ]:
# Generate predictions for all models
predictions = {}
for name, model in models.items():
    predictions[name] = model.predict(X_test)
    print(f"{name}: Generated {len(predictions[name]):,} predictions")

# Build unified error DataFrame for analysis
meta_cols = [
    "Season", "RoundNumber", "EventName", "Circuit",
    "Driver", "Team", "Compound", "TireAgeCategory",
    "LapNumber", "Stint", "TyreLife"
]
meta_cols = [c for c in meta_cols if c in test_df.columns]

error_frames = []
for name, y_pred in predictions.items():
    frame = test_df[meta_cols].copy()
    frame["model"] = name
    frame["y_true"] = y_test
    frame["y_pred"] = y_pred
    frame["error"] = y_pred - y_test
    frame["abs_error"] = np.abs(frame["error"])
    frame["pct_error"] = (frame["abs_error"] / frame["y_true"].clip(lower=1)) * 100
    error_frames.append(frame)

errors_df = pd.concat(error_frames, ignore_index=True)
print(f"\nError analysis DataFrame: {len(errors_df):,} rows")

---
## 4. Comprehensive Numerical Metrics

**This cell outputs ALL numerical metrics for the paper - no plots, just numbers.**

In [ ]:
# =============================================================================
# COMPREHENSIVE NUMERICAL METRICS (ALL MODELS)
# =============================================================================

def compute_all_metrics(y_true, y_pred, n_features):
    """Compute comprehensive metrics for a single model."""
    n = len(y_true)
    residuals = y_pred - y_true
    abs_errors = np.abs(residuals)
    
    # Avoid division by zero for percentage metrics
    y_true_safe = np.clip(np.abs(y_true), 1e-6, None)
    
    metrics = {
        # Sample size
        "n_samples": n,
        "n_features": n_features,
        
        # === PRIMARY METRICS ===
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "R2": r2_score(y_true, y_pred),
        
        # === ADJUSTED R2 ===
        "R2_adjusted": 1 - (1 - r2_score(y_true, y_pred)) * (n - 1) / (n - n_features - 1),
        
        # === PERCENTAGE METRICS ===
        "MAPE_pct": float(np.mean(abs_errors / y_true_safe) * 100),
        "sMAPE_pct": float(np.mean(2 * abs_errors / (np.abs(y_true) + np.abs(y_pred) + 1e-6)) * 100),
        "RMSPE_pct": float(np.sqrt(np.mean((abs_errors / y_true_safe) ** 2)) * 100),
        
        # === ERROR DISTRIBUTION ===
        "MedianAE": float(np.median(abs_errors)),
        "MaxError": float(np.max(abs_errors)),
        "MinError": float(np.min(abs_errors)),
        "StdError": float(np.std(residuals)),
        "MAD_error": float(np.median(np.abs(residuals - np.median(residuals)))),
        
        # === BIAS METRICS ===
        "Bias_mean": float(np.mean(residuals)),
        "Bias_median": float(np.median(residuals)),
        
        # === PERCENTILES OF ABSOLUTE ERROR ===
        "P50_abs_error": float(np.percentile(abs_errors, 50)),
        "P75_abs_error": float(np.percentile(abs_errors, 75)),
        "P90_abs_error": float(np.percentile(abs_errors, 90)),
        "P95_abs_error": float(np.percentile(abs_errors, 95)),
        "P99_abs_error": float(np.percentile(abs_errors, 99)),
        
        # === THRESHOLD ACCURACY ===
        "Within_0.5s_pct": float((abs_errors <= 0.5).mean() * 100),
        "Within_1s_pct": float((abs_errors <= 1.0).mean() * 100),
        "Within_2s_pct": float((abs_errors <= 2.0).mean() * 100),
        "Within_3s_pct": float((abs_errors <= 3.0).mean() * 100),
        "Within_5s_pct": float((abs_errors <= 5.0).mean() * 100),
        
        # === CORRELATION ===
        "Pearson_r": float(np.corrcoef(y_true, y_pred)[0, 1]),
        "Spearman_r": float(pd.Series(y_true).corr(pd.Series(y_pred), method="spearman")),
        
        # === DESCRIPTIVE STATS ===
        "Mean_actual": float(np.mean(y_true)),
        "Std_actual": float(np.std(y_true)),
        "Mean_predicted": float(np.mean(y_pred)),
        "Std_predicted": float(np.std(y_pred)),
    }
    
    return metrics


# Compute metrics for all models
all_metrics = []
for name, y_pred in predictions.items():
    m = compute_all_metrics(y_test, y_pred, len(features))
    m["Model"] = name
    all_metrics.append(m)

metrics_df = pd.DataFrame(all_metrics)

# Reorder columns
col_order = ["Model", "n_samples", "n_features"] + [c for c in metrics_df.columns if c not in ["Model", "n_samples", "n_features"]]
metrics_df = metrics_df[col_order].sort_values("MAE").reset_index(drop=True)

In [ ]:
# =============================================================================
# DISPLAY: PRIMARY METRICS
# =============================================================================
print("=" * 80)
print("PRIMARY METRICS (2025 Test Set - Unseen Data)")
print("=" * 80)

primary_cols = ["Model", "n_samples", "MAE", "RMSE", "R2", "R2_adjusted"]
display(metrics_df[primary_cols].style.format({
    "MAE": "{:.4f}",
    "RMSE": "{:.4f}",
    "R2": "{:.4f}",
    "R2_adjusted": "{:.4f}",
}))

In [ ]:
# =============================================================================
# DISPLAY: PERCENTAGE METRICS
# =============================================================================
print("\n" + "=" * 80)
print("PERCENTAGE METRICS")
print("=" * 80)

pct_cols = ["Model", "MAPE_pct", "sMAPE_pct", "RMSPE_pct"]
display(metrics_df[pct_cols].style.format({
    "MAPE_pct": "{:.2f}%",
    "sMAPE_pct": "{:.2f}%",
    "RMSPE_pct": "{:.2f}%",
}))

In [ ]:
# =============================================================================
# DISPLAY: ERROR DISTRIBUTION
# =============================================================================
print("\n" + "=" * 80)
print("ERROR DISTRIBUTION")
print("=" * 80)

dist_cols = ["Model", "MedianAE", "MAD_error", "StdError", "MinError", "MaxError"]
display(metrics_df[dist_cols].style.format({
    "MedianAE": "{:.4f}s",
    "MAD_error": "{:.4f}s",
    "StdError": "{:.4f}s",
    "MinError": "{:.4f}s",
    "MaxError": "{:.2f}s",
}))

In [ ]:
# =============================================================================
# DISPLAY: PERCENTILES OF ABSOLUTE ERROR
# =============================================================================
print("\n" + "=" * 80)
print("ABSOLUTE ERROR PERCENTILES")
print("=" * 80)

pctl_cols = ["Model", "P50_abs_error", "P75_abs_error", "P90_abs_error", "P95_abs_error", "P99_abs_error"]
display(metrics_df[pctl_cols].style.format({
    "P50_abs_error": "{:.3f}s",
    "P75_abs_error": "{:.3f}s",
    "P90_abs_error": "{:.3f}s",
    "P95_abs_error": "{:.3f}s",
    "P99_abs_error": "{:.3f}s",
}))

In [ ]:
# =============================================================================
# DISPLAY: THRESHOLD ACCURACY (% within X seconds)
# =============================================================================
print("\n" + "=" * 80)
print("THRESHOLD ACCURACY (% of predictions within X seconds)")
print("=" * 80)

thresh_cols = ["Model", "Within_0.5s_pct", "Within_1s_pct", "Within_2s_pct", "Within_3s_pct", "Within_5s_pct"]
display(metrics_df[thresh_cols].style.format({
    "Within_0.5s_pct": "{:.1f}%",
    "Within_1s_pct": "{:.1f}%",
    "Within_2s_pct": "{:.1f}%",
    "Within_3s_pct": "{:.1f}%",
    "Within_5s_pct": "{:.1f}%",
}))

In [ ]:
# =============================================================================
# DISPLAY: BIAS & CORRELATION
# =============================================================================
print("\n" + "=" * 80)
print("BIAS & CORRELATION")
print("=" * 80)

bias_cols = ["Model", "Bias_mean", "Bias_median", "Pearson_r", "Spearman_r"]
display(metrics_df[bias_cols].style.format({
    "Bias_mean": "{:.4f}s",
    "Bias_median": "{:.4f}s",
    "Pearson_r": "{:.4f}",
    "Spearman_r": "{:.4f}",
}))

In [ ]:
# =============================================================================
# BOOTSTRAP CONFIDENCE INTERVALS
# =============================================================================
print("\n" + "=" * 80)
print(f"BOOTSTRAP CONFIDENCE INTERVALS ({BOOTSTRAP_CI}% CI, n={BOOTSTRAP_N})")
print("=" * 80)

def bootstrap_confidence_intervals(y_true, y_pred, n_boot=BOOTSTRAP_N, ci=BOOTSTRAP_CI):
    """Compute bootstrap confidence intervals for key metrics."""
    rng = np.random.default_rng(SEED)
    n = len(y_true)
    
    boot_mae, boot_rmse, boot_r2 = [], [], []
    
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        yt, yp = y_true[idx], y_pred[idx]
        boot_mae.append(mean_absolute_error(yt, yp))
        boot_rmse.append(np.sqrt(mean_squared_error(yt, yp)))
        boot_r2.append(r2_score(yt, yp))
    
    alpha = (100 - ci) / 2
    return {
        "MAE_CI": f"[{np.percentile(boot_mae, alpha):.4f}, {np.percentile(boot_mae, 100-alpha):.4f}]",
        "RMSE_CI": f"[{np.percentile(boot_rmse, alpha):.4f}, {np.percentile(boot_rmse, 100-alpha):.4f}]",
        "R2_CI": f"[{np.percentile(boot_r2, alpha):.4f}, {np.percentile(boot_r2, 100-alpha):.4f}]",
    }

ci_results = []
for name, y_pred in predictions.items():
    ci = bootstrap_confidence_intervals(y_test, y_pred)
    ci["Model"] = name
    ci_results.append(ci)

ci_df = pd.DataFrame(ci_results)[["Model", "MAE_CI", "RMSE_CI", "R2_CI"]]
display(ci_df)

In [ ]:
# =============================================================================
# SAVE ALL METRICS TO CSV
# =============================================================================
metrics_df.to_csv(OUTPUT_DIR / "metrics_comprehensive_2025.csv", index=False)
print(f"\nSaved: {OUTPUT_DIR / 'metrics_comprehensive_2025.csv'}")

# Save predictions for reproducibility
errors_df.to_parquet(OUTPUT_DIR / "predictions_2025.parquet", index=False)
print(f"Saved: {OUTPUT_DIR / 'predictions_2025.parquet'}")

---
## 5. Model Comparison Plots

In [ ]:
# =============================================================================
# PLOT: MAE & RMSE Comparison
# =============================================================================
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=["Mean Absolute Error (MAE)", "Root Mean Square Error (RMSE)"],
    horizontal_spacing=0.12,
)

for i, metric in enumerate(["MAE", "RMSE"], 1):
    for _, row in metrics_df.iterrows():
        fig.add_trace(
            go.Bar(
                x=[row["Model"]],
                y=[row[metric]],
                name=row["Model"],
                marker_color=MODEL_COLORS[row["Model"]],
                text=[f"{row[metric]:.3f}s"],
                textposition="outside",
                showlegend=(i == 1),
            ),
            row=1, col=i,
        )

fig.update_yaxes(title_text="Seconds", row=1, col=1)
fig.update_yaxes(title_text="Seconds", row=1, col=2)
style_figure(fig, title="Model Comparison: Error Metrics (2025 Test Set)", height=450, width=900)
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5))
save_and_show(fig, "model_comparison_mae_rmse")

In [ ]:
# =============================================================================
# PLOT: R² Comparison
# =============================================================================
fig = go.Figure()

for _, row in metrics_df.iterrows():
    fig.add_trace(
        go.Bar(
            x=[row["Model"]],
            y=[row["R2"]],
            name=row["Model"],
            marker_color=MODEL_COLORS[row["Model"]],
            text=[f"{row['R2']:.4f}"],
            textposition="outside",
        )
    )

fig.update_yaxes(title_text="R² Score", range=[0, 1])
fig.update_xaxes(title_text="Model")
style_figure(fig, title="Model Comparison: R² Score (2025 Test Set)", height=420)
save_and_show(fig, "model_comparison_r2")

In [ ]:
# =============================================================================
# PLOT: Threshold Accuracy (% within X seconds)
# =============================================================================
thresholds = [0.5, 1.0, 2.0, 3.0, 5.0]
thresh_data = []

for name, y_pred in predictions.items():
    abs_err = np.abs(y_test - y_pred)
    for t in thresholds:
        thresh_data.append({
            "Model": name,
            "Threshold": f"<= {t}s",
            "Percentage": (abs_err <= t).mean() * 100,
        })

thresh_df = pd.DataFrame(thresh_data)

fig = px.bar(
    thresh_df,
    x="Threshold",
    y="Percentage",
    color="Model",
    barmode="group",
    color_discrete_map=MODEL_COLORS,
    text=thresh_df["Percentage"].round(1).astype(str) + "%",
)

fig.update_traces(textposition="outside")
fig.update_yaxes(title_text="% of Predictions", range=[0, 105])
fig.update_xaxes(title_text="Error Threshold")
style_figure(fig, title="Prediction Accuracy: % Within Error Thresholds (2025)", height=480)
save_and_show(fig, "accuracy_thresholds")

In [ ]:
# =============================================================================
# PLOT: Cumulative Error Distribution (CDF)
# =============================================================================
fig = go.Figure()

for name, y_pred in predictions.items():
    abs_err = np.abs(y_test - y_pred)
    sorted_err = np.sort(abs_err)
    cumulative = np.arange(1, len(sorted_err) + 1) / len(sorted_err) * 100
    
    fig.add_trace(go.Scatter(
        x=sorted_err,
        y=cumulative,
        mode="lines",
        name=name,
        line=dict(color=MODEL_COLORS[name], width=2.5),
    ))

# Reference lines
for t in [1, 2, 5]:
    fig.add_vline(x=t, line_dash="dot", line_color="gray", opacity=0.5,
                  annotation_text=f"{t}s", annotation_position="top")

fig.update_xaxes(title_text="Absolute Error (seconds)", range=[0, 10])
fig.update_yaxes(title_text="Cumulative % of Predictions")
style_figure(fig, title="Cumulative Error Distribution (2025 Test Set)", height=480)
fig.update_layout(legend=dict(x=0.75, y=0.25))
save_and_show(fig, "cumulative_error_distribution")

---
## 6. Per-Model Diagnostic Plots

In [ ]:
# =============================================================================
# PLOT: Actual vs Predicted (per model)
# =============================================================================
for name, y_pred in predictions.items():
    # Sample for visualization
    n_sample = min(5000, len(y_test))
    idx = np.random.choice(len(y_test), n_sample, replace=False)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=y_test[idx],
        y=y_pred[idx],
        mode="markers",
        marker=dict(color=MODEL_COLORS[name], opacity=0.4, size=5),
        name="Predictions",
    ))
    
    # Perfect prediction line
    min_v, max_v = min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())
    fig.add_trace(go.Scatter(
        x=[min_v, max_v],
        y=[min_v, max_v],
        mode="lines",
        line=dict(color="red", dash="dash", width=2),
        name="Perfect Prediction",
    ))
    
    fig.update_xaxes(title_text="Actual Lap Time (s)")
    fig.update_yaxes(title_text="Predicted Lap Time (s)")
    style_figure(fig, title=f"Actual vs Predicted: {name} (2025)", height=500)
    save_and_show(fig, f"actual_vs_predicted_{name.lower().replace(' ', '_')}")

In [ ]:
# =============================================================================
# PLOT: Residual Distribution (per model)
# =============================================================================
for name, y_pred in predictions.items():
    residuals = y_pred - y_test
    
    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=residuals,
        nbinsx=80,
        marker_color=MODEL_COLORS[name],
        opacity=0.75,
        name="Residuals",
    ))
    
    # Add vertical line at 0
    fig.add_vline(x=0, line_dash="dash", line_color="red", line_width=2)
    
    # Add mean annotation
    mean_res = np.mean(residuals)
    fig.add_annotation(
        x=mean_res, y=0.95, yref="paper",
        text=f"Mean: {mean_res:.3f}s",
        showarrow=True, arrowhead=2,
    )
    
    fig.update_xaxes(title_text="Residual (Predicted - Actual) [s]")
    fig.update_yaxes(title_text="Count")
    style_figure(fig, title=f"Residual Distribution: {name} (2025)", height=420)
    save_and_show(fig, f"residuals_{name.lower().replace(' ', '_')}")

In [ ]:
# =============================================================================
# PLOT: Residuals vs Predicted (per model) - Heteroscedasticity check
# =============================================================================
for name, y_pred in predictions.items():
    residuals = y_pred - y_test
    
    # Sample for visualization
    n_sample = min(5000, len(y_test))
    idx = np.random.choice(len(y_test), n_sample, replace=False)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=y_pred[idx],
        y=residuals[idx],
        mode="markers",
        marker=dict(color=MODEL_COLORS[name], opacity=0.4, size=5),
        name="Residuals",
    ))
    
    fig.add_hline(y=0, line_dash="dash", line_color="red", line_width=2)
    
    fig.update_xaxes(title_text="Predicted Lap Time (s)")
    fig.update_yaxes(title_text="Residual (s)")
    style_figure(fig, title=f"Residuals vs Predicted: {name} (2025)", height=450)
    save_and_show(fig, f"residuals_vs_pred_{name.lower().replace(' ', '_')}")

---
## 6b. XGBoost Feature Importance

Critical for model interpretability - shows which features XGBoost relies on most.

In [ ]:
# =============================================================================
# PLOT: XGBoost Feature Importance (Top 20)
# =============================================================================
if "XGBoost" in models:
    xgb_model = models["XGBoost"]
    
    # Extract XGBoost estimator from pipeline
    if hasattr(xgb_model, "named_steps"):
        xgb_estimator = xgb_model.named_steps.get("model")
    elif hasattr(xgb_model, "steps"):
        xgb_estimator = xgb_model.steps[-1][1]
    else:
        xgb_estimator = xgb_model
    
    if hasattr(xgb_estimator, "feature_importances_"):
        importances = xgb_estimator.feature_importances_
        
        # Map to feature names
        if len(importances) == len(features):
            feature_names = features
        else:
            feature_names = [f"Feature_{i}" for i in range(len(importances))]
        
        imp_df = pd.DataFrame({
            "Feature": feature_names,
            "Importance": importances
        }).sort_values("Importance", ascending=True).tail(20)
        
        # Save full importance to CSV
        full_imp = pd.DataFrame({
            "Feature": feature_names,
            "Importance": importances
        }).sort_values("Importance", ascending=False)
        full_imp.to_csv(OUTPUT_DIR / "feature_importance_xgboost.csv", index=False)
        
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=imp_df["Importance"],
            y=imp_df["Feature"],
            orientation="h",
            marker_color=MODEL_COLORS["XGBoost"],
            text=imp_df["Importance"].round(4),
            textposition="outside",
        ))
        
        fig.update_xaxes(title_text="Importance (Gain)")
        fig.update_yaxes(title_text="Feature")
        style_figure(fig, title="XGBoost Feature Importance (Top 20)", height=600)
        save_and_show(fig, "feature_importance_xgboost")
    else:
        print("XGBoost model has no feature_importances_ attribute.")
else:
    print("XGBoost model not loaded.")

---
## 7. Error Analysis by Category

In [ ]:
# =============================================================================
# ERROR BY CATEGORY: Helper Function
# =============================================================================
def plot_error_by_category(errors_df, category, model_name, top_n=12):
    """Create bar chart of MAE by category."""
    subset = errors_df[errors_df["model"] == model_name]
    
    grouped = (
        subset.groupby(category)
        .agg(
            n=("abs_error", "size"),
            mae=("abs_error", "mean"),
        )
        .sort_values("mae", ascending=False)
        .head(top_n)
        .reset_index()
    )
    
    fig = px.bar(
        grouped,
        x="mae",
        y=category,
        orientation="h",
        color_discrete_sequence=[MODEL_COLORS[model_name]],
        text=grouped["mae"].round(3).astype(str) + "s",
    )
    
    fig.update_traces(textposition="outside")
    fig.update_xaxes(title_text="MAE (seconds)")
    fig.update_yaxes(title_text=category, categoryorder="total ascending")
    
    return fig, grouped

In [ ]:
# =============================================================================
# PLOT: Error by Circuit (best model)
# =============================================================================
best_model = metrics_df.iloc[0]["Model"]
print(f"Best model by MAE: {best_model}")

if "Circuit" in errors_df.columns:
    fig, _ = plot_error_by_category(errors_df, "Circuit", best_model, top_n=15)
    style_figure(fig, title=f"MAE by Circuit: {best_model} (2025)", height=520)
    save_and_show(fig, f"mae_by_circuit_{best_model.lower().replace(' ', '_')}")

In [ ]:
# =============================================================================
# PLOT: Error by Compound (all models comparison)
# =============================================================================
if "Compound" in errors_df.columns:
    compound_data = []
    for name in models.keys():
        subset = errors_df[errors_df["model"] == name]
        for compound in subset["Compound"].unique():
            mask = subset["Compound"] == compound
            if mask.sum() >= 10:
                compound_data.append({
                    "Model": name,
                    "Compound": compound,
                    "MAE": subset.loc[mask, "abs_error"].mean(),
                    "n": mask.sum(),
                })
    
    compound_df = pd.DataFrame(compound_data)
    compound_df.to_csv(OUTPUT_DIR / "mae_by_compound_all_models.csv", index=False)
    
    fig = px.bar(
        compound_df,
        x="Compound",
        y="MAE",
        color="Model",
        barmode="group",
        color_discrete_map=MODEL_COLORS,
        text=compound_df["MAE"].round(3).astype(str) + "s",
    )
    
    fig.update_traces(textposition="outside")
    fig.update_yaxes(title_text="MAE (seconds)")
    style_figure(fig, title="MAE by Tire Compound: All Models (2025)", height=480)
    save_and_show(fig, "mae_by_compound_comparison")

In [ ]:
# =============================================================================
# PLOT: Error by Team (best model)
# =============================================================================
if "Team" in errors_df.columns:
    fig, team_metrics = plot_error_by_category(errors_df, "Team", best_model, top_n=12)
    style_figure(fig, title=f"MAE by Team: {best_model} (2025)", height=480)
    team_metrics.to_csv(OUTPUT_DIR / f"mae_by_team_{best_model.lower().replace(' ', '_')}.csv", index=False)
    save_and_show(fig, f"mae_by_team_{best_model.lower().replace(' ', '_')}")

In [ ]:
# =============================================================================
# PLOT: Error by Driver (best model)
# =============================================================================
if "Driver" in errors_df.columns:
    fig, driver_metrics = plot_error_by_category(errors_df, "Driver", best_model, top_n=12)
    style_figure(fig, title=f"MAE by Driver: {best_model} (2025)", height=520)
    driver_metrics.to_csv(OUTPUT_DIR / f"mae_by_driver_{best_model.lower().replace(' ', '_')}.csv", index=False)
    save_and_show(fig, f"mae_by_driver_{best_model.lower().replace(' ', '_')}")

---
## 8. Temporal Analysis (by Round)

In [ ]:
# =============================================================================
# METRICS BY ROUND (all models)
# =============================================================================
round_metrics = []

for name in models.keys():
    subset = errors_df[errors_df["model"] == name]
    
    for (rnd, event), grp in subset.groupby(["RoundNumber", "EventName"]):
        y_true_g = grp["y_true"].to_numpy()
        y_pred_g = grp["y_pred"].to_numpy()
        
        round_metrics.append({
            "Model": name,
            "Round": int(rnd),
            "Event": event,
            "n": len(grp),
            "MAE": mean_absolute_error(y_true_g, y_pred_g),
            "RMSE": np.sqrt(mean_squared_error(y_true_g, y_pred_g)),
            "R2": r2_score(y_true_g, y_pred_g) if len(grp) > 1 else np.nan,
        })

round_df = pd.DataFrame(round_metrics).sort_values(["Model", "Round"])
round_df.to_csv(OUTPUT_DIR / "metrics_by_round_2025.csv", index=False)

display(round_df.pivot_table(index=["Round", "Event"], columns="Model", values="MAE").round(3))

In [ ]:
# =============================================================================
# PLOT: MAE by Round (all models)
# =============================================================================
fig = px.line(
    round_df,
    x="Round",
    y="MAE",
    color="Model",
    markers=True,
    color_discrete_map=MODEL_COLORS,
    hover_data=["Event", "n"],
)

fig.update_traces(line_width=2.5, marker_size=8)
fig.update_xaxes(title_text="Round Number", dtick=1)
fig.update_yaxes(title_text="MAE (seconds)")
style_figure(fig, title="MAE by Round: All Models (2025 Season)", height=450)
save_and_show(fig, "mae_by_round_2025")

In [ ]:
# =============================================================================
# PLOT: Error vs Lap Number (race progression)
# =============================================================================
if "LapNumber" in errors_df.columns:
    subset = errors_df[errors_df["model"] == best_model]
    
    lap_metrics = (
        subset.groupby("LapNumber")
        .agg(
            mae=("abs_error", "mean"),
            std=("abs_error", "std"),
            n=("abs_error", "size"),
        )
        .reset_index()
    )
    lap_metrics = lap_metrics[lap_metrics["n"] >= 20]  # Filter low-sample laps
    
    fig = go.Figure()
    
    # Confidence band
    fig.add_trace(go.Scatter(
        x=pd.concat([lap_metrics["LapNumber"], lap_metrics["LapNumber"][::-1]]),
        y=pd.concat([lap_metrics["mae"] + lap_metrics["std"], 
                     (lap_metrics["mae"] - lap_metrics["std"])[::-1]]),
        fill="toself",
        fillcolor=f"rgba({int(MODEL_COLORS[best_model][1:3], 16)}, {int(MODEL_COLORS[best_model][3:5], 16)}, {int(MODEL_COLORS[best_model][5:7], 16)}, 0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        name="+/- 1 Std",
    ))
    
    # Mean line
    fig.add_trace(go.Scatter(
        x=lap_metrics["LapNumber"],
        y=lap_metrics["mae"],
        mode="lines+markers",
        name="MAE",
        line=dict(color=MODEL_COLORS[best_model], width=2.5),
        marker=dict(size=4),
    ))
    
    fig.update_xaxes(title_text="Lap Number")
    fig.update_yaxes(title_text="MAE (seconds)")
    style_figure(fig, title=f"Prediction Error vs Lap Number: {best_model} (2025)", height=450)
    save_and_show(fig, "error_vs_lap_number")

---
## Summary

All metrics and plots have been saved to the output directory.

In [ ]:
# =============================================================================
# LIST ALL GENERATED OUTPUTS
# =============================================================================
print("=" * 60)
print("GENERATED OUTPUTS")
print("=" * 60)
print(f"\nDirectory: {OUTPUT_DIR}\n")

csv_files = sorted(OUTPUT_DIR.glob("*.csv"))
png_files = sorted(OUTPUT_DIR.glob("*.png"))
html_files = sorted(OUTPUT_DIR.glob("*.html"))
parquet_files = sorted(OUTPUT_DIR.glob("*.parquet"))

print(f"CSV files ({len(csv_files)}):")
for f in csv_files:
    print(f"  - {f.name}")

print(f"\nPlot files ({len(png_files)} PNG, {len(html_files)} HTML):")
for f in png_files:
    print(f"  - {f.name}")

print(f"\nData files ({len(parquet_files)}):")
for f in parquet_files:
    print(f"  - {f.name}")